In [1]:
# Imports
import json
import typing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import seaborn as sns
import gc
from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud
import squarify
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re 
import gensim 
from gensim import corpora

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.style.use('bmh')

In [2]:
def json_to_df(path):
    with open(path, encoding="utf8") as data_file:
        data = []
        for line in data_file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

In [3]:
df_sg = json_to_df("C:/Users/Francis Zhou/Desktop/Data Science & Machine Learning/Semester 1/DSA5104 Principles of Data Management and Retrieval/Project/merged_sg_data.json")
df_sg.head(6)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,Business Yelp url,Business Website,Reviews
0,39d99ccd912dab21320ba84d93446da0,Bakalaki Greek Taverna,38A Martin Rd,Singapore,Singapore,239072,1.292122,103.838004,4.5,22,1,,,"{'Mon': '6:00 PM - 12:00 AM (Next day)', None:...",https://www.yelp.com/biz/bakalaki-greek-tavern...,bakalaki.com,"[{'Reviewer Name': 'Luke B.', 'Reviewer Locati..."
1,91f9d25fbc93e9889f1ced26be4768e0,Bigmama,2 Kim Tian Road,Singapore,Singapore,169244,1.285260,103.829337,3.9,17,1,,,"{'Mon': 'Closed', None: None, 'Tue': '11:30 AM...",https://www.yelp.com/biz/bigmama-singapore,bigmama.sg,"[{'Reviewer Name': 'SY C.', 'Reviewer Location..."
2,d3e4ca4bb032cde927a2452432876865,Tiong Bahru Bakery,56 Eng Hoon St,Singapore,Singapore,160056,1.284227,103.833876,3.8,69,1,,,"{'Mon': '8:00 AM - 8:00 PM', None: None, 'Tue'...",https://www.yelp.com/biz/tiong-bahru-bakery-si...,tiongbahrubakery.com,"[{'Reviewer Name': 'Weldon W.', 'Reviewer Loca..."
3,0878c28f74b1d52c98df9d11befe678d,Potato Head Folk,36 Keong Saik Road,Singapore,Singapore,089143,1.280440,103.841700,3.8,59,1,,,"{'Mon': 'Closed', None: None, 'Tue': '11:00 AM...",https://www.yelp.com/biz/potato-head-folk-sing...,pttheadfolk.com,"[{'Reviewer Name': 'Jodi H.', 'Reviewer Locati..."
4,3bfd7a7ec2e535c8cb5413bf82d3a7e9,Coq & Balls,6 Kim Tian Rd,Singapore,Singapore,169246,1.285252,103.829245,4.2,13,1,,,"{'Mon': '5:00 PM - 1:00 AM (Next day)', None: ...",https://www.yelp.com/biz/coq-and-balls-singapore,coqnballs.com,"[{'Reviewer Name': 'Krissy M.', 'Reviewer Loca..."
5,558f534defa1d30ff7a52a3f24bd88cd,Shabestan,80 Mohammed Sultan Rd,Singapore,Singapore,239013,1.290909,103.840453,4.4,24,1,,,"{'Mon': '12:00 PM - 12:00 AM (Next day)', None...",https://www.yelp.com/biz/shabestan-singapore,shabestan.sg,"[{'Reviewer Name': 'Mehr A.', 'Reviewer Locati..."


In [4]:
df_sg.shape

(6606, 17)

In [5]:
# Count of missing data for each attribute
print("Missing values count: ")
print(df_sg.isnull().sum())
print("")

Missing values count: 
business_id             0
name                    0
address                 0
city                    0
state                   0
postal_code             0
latitude                0
longitude               0
stars                   0
review_count            0
is_open                 0
attributes              0
categories              0
hours                   0
Business Yelp url       0
Business Website     3003
Reviews                 0
dtype: int64



In [ ]:
# singapore = df_sg.dropna()
# singapore

In [ ]:
# fig = plt.figure(figsize=(14, 10), edgecolor='w')
# m = Basemap(projection='cyl',llcrnrlon= -180, urcrnrlon = 180, llcrnrlat = -90, urcrnrlat= 90, resolution='c', lat_ts = True)
# m.drawcoastlines() # add coastlines
# m.fillcontinents(color='#04BAE3',lake_color='#FFFFFF') # add continents lines
# m.drawcountries() # add country border lines
# m.drawmapboundary(fill_color='#FFFFFF') # add glabe borders

# mloc = m(df_sg['latitude'].tolist(), df_sg['longitude'].tolist())
# m.scatter(mloc[1],mloc[0],color ='red',lw=5,alpha=0.25,zorder=2.5);

In [21]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

m = folium.Map(location=[1.3521, 103.8198], tiles='cartodbpositron', zoom_start=10)

mc = MarkerCluster()
for idx, row in df_sg.sample(6000).iterrows():
    mc.add_child(Marker([row['latitude'], row['longitude']]))
m.add_child(mc)
print("Map with 10k samples:")
m

Map with 10k samples:
